In [ ]:
import os

import matplotlib.pyplot as plt
from matplotlib import patches
from mpl_toolkits.mplot3d import art3d
import numpy as np
import seaborn as sns

from src.data import load_ear_pcd
from src.mappings import encode_normals_rgb
from src.metrics import angle_error, rms_angle_error
from src.normals import (estim_normals_pca,
                         estim_normals_spline,
                         estim_normals_poly)
from src.orientation import orient_normals
from src.plotting import set_defense_context, set_axes_equal, draw_unit_cube

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
# constants

c = sns.color_palette('rocket', n_colors=1)
elev, azim = 20, 155

In [ ]:
def add_coordinate_frame(ax):
    """Set RGB coordinate frame to axes.
    
    Parameters
    ----------
    ax : matplotlib.axes._subplots.Axes3DSubplot
        3-D axes subplot.
    
    Returns
    -------
    matplotlib.axes._subplots.Axes3DSubplot
        Axes with coordinate frame.
    """
    ax.quiver(-1.5, -1, -1.5, 0.75, 0, 0, lw=0.5, color='r')
    ax.text(0, -1, -1.5, s='$x$', color='r', fontweight='bold')
    ax.quiver(-1.5, -1, -1.5, 0, 0.75, 0, lw=0.5, color='g')
    ax.text(-1.5, +0.25, -1.5, s='$y$', color='g', fontweight='bold')
    ax.quiver(-1.5, -1, -1.5, 0, 0, 0.75, lw=0.5, color='b')
    ax.text(-1.5, -1, -0.5, s='$z$', color='b', fontweight='bold')
    ax.scatter(-1.5, -1, -1.5, s=7, color='k', depthshade=False)
    return ax

# Data

In [ ]:
xyz = load_ear_pcd() * 100  # in cm

# PCA via Open3D - unit normal

In [ ]:
# estimate unit normals

n_pca = estim_normals_pca(xyz, k=100)
n_pca = orient_normals(xyz, n_pca, k=30)

In [ ]:
# grid downsample

N = xyz.shape[0]
num = 0.05 * N
mask = np.arange(0, N, int(N/num))

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.scatter(*xyz[mask].T, color='k', fc='w', ec='k', s=7, lw=0.5)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'cloud.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

In [ ]:
xi = xyz[np.argmin(xyz[mask, 0]), ...]
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.scatter(*xyz[mask].T, fc='w', ec='k', s=7, lw=0.5)
    ax.plot(*xi, marker='o', ls='none', mfc=c[0], mec='k', mew=0.5, ms=4)
    p = patches.Circle(xi[1:], 0.5, fc='none', ec='k', lw=1.5)
    ax.add_patch(p)
    art3d.pathpatch_2d_to_3d(p, z=0, zdir='x')
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'cloud-xi.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

In [ ]:
xyz[mask].shape

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax.quiver(*xyz[mask].T, *n_pca[mask].T, color='k',
              length=0.55, lw=0.5)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'normals.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

In [ ]:
# convert arrows to rgb cube

with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, color=encode_normals_rgb(n_pca), s=0.3)
    ax = add_coordinate_frame(ax)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'normals-rgb.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

# Bivariate B-spline - surface (curvature) normal

In [ ]:
# estimate surface normals

n_spl = estim_normals_spline(xyz, k=100, deg=3, s=1)
n_spl = orient_normals(xyz, n_spl, k=30)

In [ ]:
# convert arrows to rgb cube

with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, color=encode_normals_rgb(n_spl), s=0.3)
    ax = add_coordinate_frame(ax)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'surface-normals.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

# Error angle

In [ ]:
# compute error wrt orientation

theta = angle_error(n_spl, n_pca)
theta_rms = rms_angle_error(n_spl, n_pca)
theta_rms

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 3))
    ax = plt.axes()
    ax.hist(theta, bins=31, density=True, ec='k', fc=c[0])
    ax.set(xlabel=r'$\theta$ (°)', ylabel='density',
           title=f'rms($\\theta$) = {theta_rms:.2f}°')
    fig.tight_layout()
    sns.despine()
    plt.show()
    fname = 'error-hist.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

In [ ]:
theta = angle_error(n_spl, n_pca, orient=False)
mask = np.where(theta <= np.quantile(theta, 0.99))[0]

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(4, 4), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz[mask].T, c=theta[mask], s=0.3)
    ax = add_coordinate_frame(ax)
    fig.colorbar(s, ax=ax, orientation='vertical', shrink=0.5, pad=-0.25,
                 label=r'$\theta$ (°)')
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'error-dist.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')

# RGB cube

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(2, 2), constrained_layout=True)
    ax = plt.axes(projection ='3d')
    ax = draw_unit_cube(ax)
    ax.view_init(elev, azim)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    plt.show()
    fname = 'rgb-cube.png'
    fig.savefig(os.path.join('figures', 'ear', fname),
                dpi=500, bbox_inches='tight')